In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

def load_data():
    # Load the datasets (Modify paths if needed)
    movies = pd.read_csv('/content/drive/MyDrive/Data Mining/movies.csv')
    ratings = pd.read_csv('/content/drive/MyDrive/Data Mining/ratings.csv')

    return movies, ratings

def create_movie_matrix(ratings):
    movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
    return movie_matrix

def compute_similarity(movie_matrix):
    similarity_matrix = cosine_similarity(movie_matrix.fillna(0).T)
    return pd.DataFrame(similarity_matrix, index=movie_matrix.columns, columns=movie_matrix.columns)

def get_similar_movies(movie_id, similarity_matrix, movies, top_n=5):
    if movie_id not in similarity_matrix:
        return []
    similar_scores = similarity_matrix[movie_id].sort_values(ascending=False)[1:top_n+1]
    return movies[movies['movieId'].isin(similar_scores.index)]

def get_personalized_recommendations(user_id, ratings, similarity_matrix, movies, top_n=5):
    user_ratings = ratings[ratings['userId'] == user_id]
    top_movies = user_ratings.nlargest(3, 'rating')['movieId']

    unseen_movies = set(movies['movieId']) - set(user_ratings['movieId'])

    recommendations = {}
    for movie in top_movies:
        if movie in similarity_matrix:
            similar_movies = similarity_matrix[movie].dropna().sort_values(ascending=False)
            for sim_movie, score in similar_movies.items():
                if sim_movie in unseen_movies:
                    recommendations[sim_movie] = recommendations.get(sim_movie, 0) + score

    best_recommendations = sorted(recommendations, key=recommendations.get, reverse=True)[:top_n]
    return movies[movies['movieId'].isin(best_recommendations)]

if __name__ == "__main__":
    movies, ratings = load_data()
    movie_matrix = create_movie_matrix(ratings)
    similarity_matrix = compute_similarity(movie_matrix)

    movie_id = 1  # Example movie ID
    print("Movies similar to the given movie:")
    print(get_similar_movies(movie_id, similarity_matrix, movies))

    user_id = 1  # Example user ID
    print("Personalized recommendations for the user:")
    print(get_personalized_recommendations(user_id, ratings, similarity_matrix, movies))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-22acf495f6f6>:17: PerformanceWarning: The following operation may generate 16966441536 cells in the resulting pandas object.
  movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
